# **Setting-up Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import keras
import joblib
import tensorflow as tf

# **Data Preprocessing**

Importing data

In [ ]:
from google.colab import files

uploaded = files.upload()

Loading data 

In [ ]:
data_train=pd.read_csv("/content/train.csv")

Verifying Data

In [ ]:
data_train.head()

In [ ]:
data_train.shape

Assigning numbers for category features

In [ ]:
data_train.category[data_train.category == 'A'] = 1
data_train.category[data_train.category == 'B'] = 2
data_train.category[data_train.category == 'C'] = 3
data_train.category[data_train.category == 'D'] = 4
data_train.category[data_train.category == 'E'] = 5
data_train.category[data_train.category == 'F'] = 6
data_train.category[data_train.category == 'G'] = 7
data_train.category[data_train.category == 'H'] = 8
data_train.head()

In [ ]:
#category={'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'H':8}
#data_train["category"]=data_train["category"].map(category)
#data_train.head()

In [ ]:
data_train=data_train[data_train.views!='F']
data_train=data_train[data_train.likes!='F']
data_train=data_train[data_train.dislikes!='F']
data_train=data_train[data_train.comment!='F']

convert string values to integers

In [ ]:
data_train["views"]=pd.to_numeric(data_train["views"])
data_train["comment"]=pd.to_numeric(data_train["comment"])
data_train["likes"]=pd.to_numeric(data_train["likes"])
data_train["dislikes"]=pd.to_numeric(data_train["dislikes"])
data_train["adview"]=pd.to_numeric(data_train["adview"])

In [ ]:
colum_vidid=data_train['vidid']

Encoding non-numerical features

In [ ]:
from sklearn.preprocessing import LabelEncoder
data_train['duration']=LabelEncoder().fit_transform(data_train['duration'])
data_train['vidid']=LabelEncoder().fit_transform(data_train['vidid'])
data_train['published']=LabelEncoder().fit_transform(data_train['published'])

In [ ]:
data_train.head()

Coverting time in seconds

In [ ]:
def checki(x):
    y = x[2:]
    h = ''
    m = ''
    s = ''
    mm = ''
    P = [ 'H','M','S']
    for i in y:
        if i not in P:
            mm+=i
        else:
            if (i=="H"):
                h = mm
                mm = ''
            elif (i == "M"):
                m = mm
                mm = ''
            else:
                s = mm 
                mm = ''
    if (h==''):
        h = '00'
    if (m==''):
        m = '00'
    if (s==''):
        s = '00'
    bp = h+':'+m+':'+s
    return bp
train=pd.read_csv("train.csv")
mp = pd.read_csv("/content/train.csv")["duration"]
time = mp.apply(checki)

def func_sec(time_string):
    h,m,s = time_string.split(':')
    return int (h) * 3600 + int (m) * 60 + int(s)

time1=time.apply(func_sec)
data_train["duration"]=time1
data_train.head()

# **DATA VISUALIZATION**

Individual plots


In [ ]:
plt.hist(data_train["category"])
plt.show
plt.hist(data_train["adview"])
plt.show

Remove videos with adview > 2000000

In [ ]:
data_train=data_train[data_train["adview"]<2000000]

In [ ]:
plt.hist(data_train["category"])
plt.show
plt.hist(data_train["adview"])
plt.show

HeatMap

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = data_train.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True, ax=ax,annot=True)
plt.show()

Split Data

In [ ]:
Y_train = pd.DataFrame(data = data_train.iloc[:, 1].values, columns = ['target'])
data_train=data_train.drop(["adview"],axis=1)
data_train=data_train.drop(["vidid"],axis=1)
data_train.head()

X_train, X_test, y_train, y_test = train_test_split(data_train, Y_train, test_size=0.2, random_state=42)

X_train.shape


Normalise Data

In [ ]:
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)
X_train.mean()

Evaluation matrices

In [ ]:
def print_error(X_test, y_test, model_name):
    prediction = model_name.predict(X_test)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, prediction))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, prediction))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))


Linear Regression

In [ ]:
linear_regression = linear_model.LinearRegression()
linear_regression.fit(X_train, y_train)
print_error(X_test,y_test, linear_regression)

Support Vector Regressor

In [ ]:
supportvector_regressor = SVR()
supportvector_regressor.fit(X_train,y_train)
print_error(X_test,y_test, linear_regression)


Decision Tree Regressor

In [ ]:
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
print_error(X_test,y_test, decision_tree)

Random Forest Regressor


In [ ]:
n_estimators = 200
max_depth = 25
min_samples_split=15
min_samples_leaf=2
random_forest = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, min_samples_split=min_samples_spli
random_forest.fit(X_train,y_train)
print_error(X_test,y_test, random_forest)

## **Training ANN**

Artificial Neural Network

In [ ]:
from keras.layers import Dense
ann = keras.models.Sequential([
Dense(6, activation="relu",
input_shape=X_train.shape[1:]),
Dense(6,activation="relu"),
Dense(1)
])
optimizer=keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])
history=ann.fit(X_train,y_train,epochs=100)
ann.summary()
print_error(X_test,y_test,ann)